In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from tqdm.notebook import tqdm

from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter
import qiskit.quantum_info as qi

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Modelo de enlaces fuertes

<div>
<img src="attachment:tight-binding.png" width="400"/>
</div>

El modelo de enlaces fuertes (del inglés *tight-binding*), es un esquema en mecánica cuántica utilizado para describir la conductancia de los electrones en materiales de estado sólido. En este modelo, se trata cada átomo como un sitio de la red cristalina y las partículas necesitan una energía  $\epsilon_i$  para ocupar ese sitio. El electrón puede saltar de un sitio a un sitio vecino pagando un costo energético  $J$. El hamiltoniano siguiente describe este modelo:

$$H_{\rm tb}/\hbar = \sum_i \epsilon_i Z_i + J \sum_{\langle i,j \rangle} (X_i X_j + Y_i Y_j)$$


El primer término en el hamiltoniano describe la energía para permanecer en un sitio, el segundo término describe la energía de interacción entre sitios vecinos de la red cristalina. En presencia del potencial periódico de la red, la función de onda de una partícula cuántica cubre también los sitios vecinos, produciendo funciones de onda de Bloch extendidas. En una red cristalina uniforme, donde todos los sitios son iguales ($\epsilon_i=0$), la propagación del electrón es lineal en el tiempo y está descrita de manera continua en el tiempo por una caminata aleatoria cuántica. Contrario a el transporte difusivo clásico, donde la propagación es cuadráticamente más lenta en el tiempo.

El hamiltoniano del sistema nos permite determinar cómo el estado cuántico evoluciona en el tiempo. Esta evolución temporal está dada por la ecuación de Schrödinger:


$$ i \hbar \frac{\partial}{\partial t}|\psi(t)\rangle = H |\psi(t)\rangle $$

En el caso de un hamiltoniano independiente del tiempo (un hamiltoniano que no cambia en el tiempo), la solución de la ecuación de Schrödinger toma la forma siguiente:

$$ |\psi(t)\rangle = e^{-i H t / \hbar} |\psi(0)\rangle $$

En este ejercicio estudiaremos la dinámica temporal sujeta a un hamiltoniano de enlaces fuertes, y construiremos un circuito trotterizado para estudiar la evolución temporal.

## 1. 1. Evolución temporal de un hamiltoniano de enlaces fuertes
Primero, consideremos la evolución sujeta a un hamiltoniano de enlaces fuertes con 3 sitios. Aquí fijaremos  $J=1$, y $\epsilon_i=0$  para una red cristalina uniforme.

In [ ]:
# Importar operadores de Pauli  (I, X, Y, Z)
from qiskit.opflow import I, X, Y, Z

J = 1

#  Hamiltoniano de enlaces fuertes
def H_tb():
    # Interacciones (I es la matriz identidad; X y Y son matrices de Pauli; ^ es el producto tensorial)
    XXs = (I^X^X) + (X^X^I)
    YYs = (I^Y^Y) + (Y^Y^I)
    
    # Sumar interacciones
    H = J*(XXs + YYs)
    
    # Devuelve el hamiltoniano
    return H

In [ ]:
#  Evolución unitaria bajo un hamiltoniano de enlaces fuertes
def U_tb(t):
    H = H_tb()
    return (t * H).exp_i()

Prepararemos el estado de nuestro sistema en  $|100\rangle$ , y seguiremos las probabilidades de los estados  $|100\rangle, |010\rangle, |001\rangle$. Estos valores corresponden a la probabilidad de encontrar una partícula en cada sitio de nuestra red cristalina.

In [ ]:
# Importar estados qubit Zero (|0>) y One (|1>)
from qiskit.opflow import Zero, One

# Definir arreglo de valores de tiempo
ts = np.linspace(0, 3, 100) # NO MODIFICAR

initial_state=One^Zero^Zero

state_t=[U_tb(float(t)) @ initial_state for t in ts]

p_100= [np.abs( (~(One^Zero^Zero) @ state).eval() )**2 for state in state_t]
p_010= [np.abs( (~(Zero^One^Zero) @ state).eval() )**2 for state in state_t]
p_001= [np.abs( (~(Zero^Zero^One) @ state).eval() )**2 for state in state_t]

plt.figure(facecolor='white')
plt.plot(ts, p_100, label=r'$p_{100}$')
plt.plot(ts, p_010, label=r'$p_{010}$')
plt.plot(ts, p_001, label=r'$p_{001}$')
plt.xlabel(r'Time (1/J)')
plt.ylabel(r'Population')
plt.legend()
plt.show()

## 2. Trotterización

Para ejecutar la evolución unitaria en el tiempo en una computadora cuántica basada en el [modelo de circuitos](https://qiskit.org/documentation/apidoc/circuit.html), debemos descomponer  $U_{\text{tb}}(t)$  en un producto de compuertas de uno o dos qubits que sean nativas a la computadora cuántica. Un método para lograr esto, es la [trotterización](https://en.wikipedia.org/wiki/Hamiltonian_simulation#Product_Formulas), también conocida como las descomposición Trotter-Suzuki. Más abajo, mostramos un ejemplo de trotterización como está explicado en \[1-2\]. Como los [operadores de Pauli no conmutan](https://en.wikipedia.org/wiki/Pauli_matrices#Commutation_relations) entre ellos, la exponencial $U_{\text{tb}}(t)$ no puede dividirse en un producto de exponenciales más simples. Sin embargo, podemos aproximar $U_{\text{tb}}(t)$ como un producto de exponenciales más simples a través de la trotterización. Consideremos un subsistema de 2 partículas de espín-1/2 dentro de un sistema más grande de 3 espínes. El hamiltoniano que actúa sobre los espínes $i$ y $j$ ($i,j \in \{0,1,2\}$) sería $H^{(i,j)}_{\text{tb}} = \sigma_x^{(i)}\sigma_x^{(j)} + \sigma_y^{(i)}\sigma_y^{(j)} + \sigma_z^{(i)}\sigma_z^{(j)}$. Reescribiendo $U_{\text{tb}}(t)$ en términos de dos posibles subsistemas dentro del sistema total con $N=3$, podemos simular
​
$$
U_{\text{tb}}(t) = \exp\left[-i t \left(H^{(0,1)}_{\text{tb}} + H^{(1,2)}_{\text{tb}} \right)\right].
$$

​
$H^{(0,1)}_{\text{tb}}$ y $H^{(1,2)}_{\text{tb}}$ no conmutan, así que $U_{\text{tb}}(t) \neq \exp\left(-i t H^{(0,1)}_{\text{tb}}\right) \exp\left(-i t H^{(1,2)}_{\text{tb}} \right)$. Pero, esta descompoción en productos puede aproximarse con la  trotterización, que dice que $U_{\text{tb}}(t)$ es aproximadamente una evolución corta de
 $H^{(0,1)}_{\text{tb}}$ (tiempo = $t/n$) seguida de la evolución corta de $H^{(1,2)}_{\text{tb}}$ (tiempo = $t/n$) repetidas $n$ veces

$$
\begin{align}
U_{\text{tb}}(t) &= \exp\left[-i t \left(H^{(0,1)}_{\text{tb}} + H^{(1,2)}_{\text{tb}} \right)\right] \\
U_{\text{tb}}(t) &\approx \left[\exp\left(\dfrac{-it}{n}H^{(0,1)}_{\text{tb}}\right) \exp\left(\dfrac{-it}{n}H^{(1,2)}_{\text{tb}} \right)\right]^n.
\end{align}
$$

$n$ es el número de pasos de Trotter, y mientras $n$ aumenta, la aproximación se hace más exacta. (Notar que la manera de dividir el operador unitario en subsistemas para la trotterización no es necesariamente única). La descomposición va más allá. En cada subsistema de 2 espines, los pares de operadores de Pauli ($\sigma_x^{(i)}\sigma_x^{(j)}$, $\sigma_y^{(i)}\sigma_y^{(j)}$, y $\sigma_z^{(i)}\sigma_z^{(j)}$) conmutan. Esto significa que podemos descomponer la exponencial del hamiltoniano de un subsistema ($H^{(i,j)}_{\text{tb}}$) en productos de exponenciales más simples acercándonos más a la implementación exacta de la compuerta  $U_{\text{tb}}(t)$
​

$$
\begin{align}
U_{\text{tb}}(t) &\approx \left[\exp\left(\dfrac{-it}{n}H^{(0,1)}_{\text{tb}}\right) \exp\left(\dfrac{-it}{n}H^{(1,2)}_{\text{tb}} \right)\right]^n \\
U_{\text{tb}}(t) &\approx \left[\exp\left(\dfrac{-it}{n}\left(\sigma_x^{(0)}\sigma_x^{(1)} + \sigma_y^{(0)}\sigma_y^{(1)} \right)\right) \exp\left(\dfrac{-it}{n}\left(\sigma_x^{(1)}\sigma_x^{(2)} + \sigma_y^{(1)}\sigma_y^{(2)} \right)\right)\right]^{n} \\
U_{\text{tb}}(t) &\approx \left[\exp\left(\dfrac{-it}{n}\sigma_x^{(0)}\sigma_x^{(1)}\right) \exp\left(\dfrac{-it}{n}\sigma_y^{(0)}\sigma_y^{(1)}\right) \exp\left(\dfrac{-it}{n}\sigma_x^{(1)}\sigma_x^{(2)}\right) \exp\left(\dfrac{-it}{n}\sigma_y^{(1)}\sigma_y^{(2)}\right) \right]^{n}
\end{align}
$$

De manera simple, y para utilizar una notación más común, llamemos a los productos $XX(2t) = \exp\left(-it \sigma_x\sigma_x\right)$ y $YY(2t) = \exp\left(-it \sigma_y\sigma_y\right)$, y podemos reescribir la $U_{\text{tb}}(t)$ trotterizada

$$
U_{\text{tb}}(t) \approx \left[XX\left(\frac{2t}{n}\right)^{(0,1)} YY\left(\frac{2t}{n}\right)^{(0,1)}  XX\left(\frac{2t}{n}\right)^{(1,2)} YY\left(\frac{2t}{n}\right)^{(1,2)}\right]^{n}
$$
¡Esto es todo! Podemos descomponer $U_{\text{tb}}(t)$ aproximadamente en compuertas $XX(t)$ y $YY(t)$ de dos qubits. Estas compuertas no son nativas de qubits superconductores, pero en la sección 2, serán descompuestas en compuertas nativas de uno y dos qubits, *más detalles en el material complementario.* 
​


\[1\] Y. Salathe, et al., *Digital Quantum Simulation of Spin Models with Circuit Quantum Electrodynamics*, [Phys. Rev. X **5**, 021027 (2015)](https://link.aps.org/doi/10.1103/PhysRevX.5.021027)

\[2\] F. Tacchino, et al., *Quantum Computers as Universal Quantum Simulators: State-of-the-Art and Perspectives*, [Adv. Quantum Technol. *3* 3 (2020)](https://doi.org/10.1002/qute.201900052) \[[free arxiv version](https://arxiv.org/abs/1907.03505)\]


### 2.1  Construyendo operaciones unitarias de Pauli individuales

En esta sección construiremos las operaciones ZZ(t), XX(t), y YY(t) usando compuertas de uno y dos qubits.

In [ ]:
t = Parameter('t')

In [ ]:
# Construye un subcircuito para ZZ(t) usando compuertas de uno a dos qubits

ZZ_qr = QuantumRegister(2)
ZZ_qc = QuantumCircuit(ZZ_qr, name='ZZ')

ZZ_qc.cnot(0,1)
ZZ_qc.rz(2 * t, 1)
ZZ_qc.cnot(0,1)

# Transforma un circuito cuántico personalizado en una compuerta
ZZ = ZZ_qc.to_instruction()

ZZ_qc.draw()

### 2.2 Compuertas de Clifford
Las compuertas de Clifford son operadores cuánticos que tranducen operadores de Pauli en otros operadores de Pauli. La compuerta Hadamard ($H$) y la compuerta de fase ($S$) son ejemplos de compuertas de Clifford de un solo qubit.

$$H=\frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\1 & -1\\ \end{pmatrix}$$

$HZH^\dagger=X$, and $HXH^\dagger=Z$. Como la compuerta Hadamard es un operador hermítico, $H=H^\dagger$.

$$S=\begin{pmatrix} 1 & 0 \\0 & i\\ \end{pmatrix}$$

$SXS^\dagger=Y$, and $SYS^\dagger=-X$.

Usando compuertas de Clifford, podemos transformar $e^{i ZZ t}$ en $e^{i XX t}$  y $e^{i YY t}$

<div class="alert alert-block alert-danger">
    
<b>Desafío, pregunta 1a</b> 

Construye un subcircuito para XX(t) usando compuertas de uno y dos qubits
    
</div>

In [ ]:

XX_qr = QuantumRegister(2)
XX_qc = QuantumCircuit(XX_qr, name='XX')

###EDITA EL CODIGO DEBAJO (añade operador de Clifford)


###NO EDITES DEBAJO DE ESTA LÍNEA


XX_qc.append(ZZ, [0,1])

###EDITA EL CODIGO DEBAJO (añade operador de Clifford)

###NO EDITES DEBAJO DE ESTA LÍNEA

# Transforma un circuito cuántico personalizado en una compuerta
XX = XX_qc.to_instruction()

XX_qc.draw()

In [ ]:
## Confirma y envía tu solución
from qc_grader.challenges.spring_2022 import grade_ex1a

grade_ex1a(XX_qc)


<div class="alert alert-block alert-danger">
    
<b>Desafío, pregunta 1b</b> 

Construye un subcircuito para YY(t) usando compuertas de uno y dos qubits
    
</div>

In [ ]:

YY_qr = QuantumRegister(2)
YY_qc = QuantumCircuit(YY_qr, name='YY')

###EDITA EL CODIGO DEBAJO (añade operador de Clifford)

###NO EDITES DEBAJO DE ESTA LÍNEA

YY_qc.append(ZZ, [0,1])

###EDITA EL CODIGO DEBAJO (añade operador de Clifford)

###NO EDITES DEBAJO DE ESTA LÍNEA

# Transforma un circuito cuántico personalizado en una compuerta
YY = YY_qc.to_instruction()

YY_qc.draw()

In [ ]:
## Confirma y envía tu solución
from qc_grader.challenges.spring_2022 import grade_ex1b

grade_ex1b(YY_qc)


### 2.3 Construyendo el circuito trotterizado

Los operadores $X_iX_j$ y $Y_iY_j$ conmutan:

$$[X_iX_j, Y_iY_j]= X_iX_j.Y_iY_j - Y_iY_j.X_iX_j = Z_iZ_j-(-Z_i)(-Z_j)=0$$.

Por lo tanto, podemos descomponer $e^{i t (X_iX_j + Y_iY_j)}$ como  $e^{i t X_iX_j} e^{i t Y_iY_j}$. Además, si $i\neq j \neq k \neq l$ entonces $[X_iX_j,X_kX_l]=0$. Basándonos en esto, podemos descomponer cada paso temporal de la trotterización en dos bloques:

$$U(\Delta t) \approx \Big(\prod_{i \in \rm{impar}} e^{-i \Delta t X_iX_{i+1}} e^{-i \Delta t Y_iY_{i+1}} \Big)  \Big(\prod_{i \in \rm{par}} e^{-i \Delta t X_iX_{i+1}} e^{-i \Delta t Y_iY_{i+1}} \Big)$$

In [ ]:
num_qubits = 3

Trot_qr = QuantumRegister(num_qubits)
Trot_qc = QuantumCircuit(Trot_qr, name='Trot')

for i in range(0, num_qubits - 1):
    Trot_qc.append(YY, [Trot_qr[i], Trot_qr[i+1]])
    Trot_qc.append(XX, [Trot_qr[i], Trot_qr[i+1]])

# Transforma un circuito cuántico personalizado en una compuerta
Trot_gate = Trot_qc.to_instruction()

Trot_qc.draw()

<div class="alert alert-block alert-danger">
    
<b>Desafío, pregunta 1c</b> 

Crea el circuito trotterizado y consigue el operador unitario asociado con el circuito
    
</div>

In [ ]:

def U_trotterize(t_target, trotter_steps):
    qr = QuantumRegister(3)
    qc = QuantumCircuit(qr)

    ###EDITA EL CÓDIGO DEBAJO (Crea un circuito trotterizado con múltiples pasos de trotter)

    
    ###NO EDITES DEBAJO DE ESTA LÍNEA
        
    qc = qc.bind_parameters({t: t_target/trotter_steps})
    
    return qi.Operator(qc)

In [ ]:
t_target = 0.5
U_target = U_tb(t_target)

steps=np.arange(1,101,2)  ## NO MODIFICAR

fidelities=[]
for n in tqdm(steps):
    U_trotter = U_trotterize(t_target, trotter_steps=n)
    fidelity = qi.process_fidelity(U_trotter, target=U_target)
    fidelities.append(fidelity)

plt.figure(facecolor='white')
plt.loglog(steps, 1-np.array(fidelities))
plt.ylabel('Trotter error')
plt.xlabel('Trotter steps')
plt.show()

In [ ]:
## Confirma y envía tu solución
from qc_grader.challenges.spring_2022 import grade_ex1c

grade_ex1c(fidelities)


# Información adicional

Traducción por: Mauricio Gómez Viloria